In [ ]:
import pandas
import lithops
import io

my_id = input()
print("Your ID is", my_id)
read_bucket_name = 'scipy-tutorial-data'
write_bucket_name = 'scipy-tutorial-' + my_id

In [ ]:
## Step 2: Define the function to drop rows with NaN values in column "value"
def drop_less_1_mile(obj):
    df = pandas.read_csv(io.BytesIO(obj.data_stream.read()))

    df = df[df['trip_distance'] >= 1]
    
    output = io.BytesIO()
    df.to_csv(output, index=False)
    output.seek(0)
    storage = lithops.storage.Storage()
    storage.put_object(write_bucket_name, obj.key.replace("etl-challenge/", "etl-challenge-processed/"), output)

In [ ]:
# Step 3: Create a Lithops executor and map the function to the data
executor = lithops.FunctionExecutor()
fs = executor.map(drop_less_1_mile, read_bucket_name + "/etl-challenge/")
executor.wait()

In [ ]:
# Step 4: Show the plots
executor.plot(fs=fs, dst=".")